In [1]:
import os
import sys
from datatools.analyzer import *

from datatools.maneger import DataManager
import pandas as pd

import csv
import time

import random
random.seed(0)

from transformer_model import *

/home/yamada/.local/share/virtualenvs/twichAI-ydQv36PI/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
out_path = "../corpus/novel_formated/"
# corpus_name = "novel_segments2.tsv"
corpus_name = "ntt_segment.tsv"

In [3]:
import csv
conv_data = []
with open(out_path+corpus_name, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    conv_data = [row for row in reader]

In [4]:
# 状況，過去発話が追加された学習データを構築
import copy
def make_context_added_Src_Tgt(conv_data):

    context_src_str = []
    tgt_str = []
    prev_utt = []
    current_situation = [""]
    for conv in conv_data:
        # 状況が変化したか
        if current_situation[0] != conv[0]:
            current_situation = conv[:-2]
            # エラー対策
            if current_situation==[]:
                current_situation = [""]
            prev_utt = [conv[-2]]
        
        context_src_str.append([current_situation, copy.deepcopy(prev_utt) ])
        prev_utt.append(conv[-1])
        tgt_str.append(conv[-1])
    
    return context_src_str, tgt_str

In [5]:
def make_Ntt_src_tgt(conv_data):

    context_src_str = []
    tgt_str = []
    
    for conv in conv_data:
        context_src_str.append(conv[:-1])
        tgt_str.append(conv[-1])
        # doc = nlp
        # tgt_str.append()
    return context_src_str, tgt_str

In [6]:
lim=5000
# src_str, tgt_str = make_context_added_Src_Tgt(conv_data)
src_str, tgt_str = make_Ntt_src_tgt(conv_data)

In [7]:
src_str[:5]

[['こんにちは。お元気ですか?', 'はい、元気です。広告代理店での仕事が忙しいですが。'],
 ['はい、元気です。広告代理店での仕事が忙しいですが。', 'お疲れ様です。私は介護福祉士をしています。'],
 ['お疲れ様です。私は介護福祉士をしています。', '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。'],
 ['介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。', 'いえ。広告代理店も大変ですよね。因みに私は北海道の一軒家に住んでいます。'],
 ['いえ。広告代理店も大変ですよね。因みに私は北海道の一軒家に住んでいます。',
  '北海道ですか。御飯の美味しい所で羨ましいです。私は青森県出身で山の多い所が好きなので、山の近くに住んでいます']]

In [8]:
tokenizer_src = mecab_tokenize
tokenizer_tgt = mecab_tokenize

In [10]:
src_trainval_str, src_test_str, tgt_trainval_str, tgt_test_str= train_test_split(src_str, tgt_str, test_size=0.20, random_state=5)
print("len=> src_train_val:{0}, src_test:{1}".format(len(src_trainval_str), len(src_test_str)))

src_train_str, src_val_str, tgt_train_str, tgt_val_str= train_test_split(src_trainval_str, tgt_trainval_str, test_size=0.10, random_state=5)

print("len=> src_train:{0}, src_val:{1}".format(len(src_train_str), len(src_val_str)))

len=> src_train_val:73397, src_test:18350
len=> src_train:66057, src_val:7340


In [120]:
src_train_str_ = src_train_str[:lim]
tgt_train_str_ = tgt_train_str[:lim]
src_val_str_ = src_val_str[:lim//10]
tgt_val_str_ = tgt_val_str[:lim//10]

In [121]:
def tgt_first_sentence(tgt):
    tgt_new = []
    for t in tqdm(tgt):
        sentences = list(nlp(t).sents)
        if len(sentences) == 0:
            tgt_new.append(t)
        else:
            tgt_new.append(sentences[0].orth_)

    return tgt_new

In [122]:
tgt_train_str_ = tgt_first_sentence(tgt_train_str_)
tgt_val_str_ = tgt_first_sentence(tgt_val_str_)

100%|██████████| 500/500 [00:05<00:00, 91.52it/s]


In [123]:
from collections import Counter
from torchtext.vocab import Vocab

def build_vocab(texts, tokenizer):
    
    counter = Counter()
    for text in tqdm(texts):
        try:
            counter.update(tokenizer(str(text)))
        except Exception as e:
            print(e)
            print(text)
    return Vocab(counter, specials=['<unk>', '<pad>', '<fos>', '<eos>','<sep>', '<cxt>', '<del>'])

In [124]:
src_train_str_set = list(set(sum(src_train_str_, [])))

In [125]:
tokenizer_tgt("難病でね。")

['難病', 'で', 'ね', '。']

In [126]:
vocab_src = build_vocab(src_train_str_set, tokenizer=tokenizer_src)
vocab_tgt = build_vocab(tgt_train_str_, tokenizer=tokenizer_tgt)

100%|██████████| 5000/5000 [00:00<00:00, 36467.26it/s]


In [127]:
vocab_path = "../models/vocab/"
vocab_name = "vocab_CModel_src_mini_lim={0}.pickle".format(lim)
dictM = DataManager(vocab_path)
dictM.save_data(vocab_name, vocab_src)
vocab_name = "vocab_CModel_tgt_mini_lim={0}.pickle".format(lim)
dictM.save_data(vocab_name, vocab_tgt)

success save : ../models/vocab/vocab_CModel_src_mini_lim=5000.pickle
success save : ../models/vocab/vocab_CModel_src_mini_lim=5000.pickle
success save : ../models/vocab/vocab_CModel_tgt_mini_lim=5000.pickle
success save : ../models/vocab/vocab_CModel_tgt_mini_lim=5000.pickle


In [128]:
PAD_IDX = vocab_src.stoi["<pad>"]
PAD_IDX

1

In [129]:
def data_process(texts_src, texts_tgt, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt):
    
    data = []
    for (src, tgt) in zip(texts_src, texts_tgt):
        src_tensor = torch.tensor(
            convert_text_to_indexes_ntt(text=src, vocab=vocab_src, tokenizer=tokenizer_src, mode="src"),
            dtype=torch.long
        )
        tgt_tensor = torch.tensor(
            convert_text_to_indexes_ntt(text=tgt, vocab=vocab_tgt, tokenizer=tokenizer_tgt, mode="tgt"),
            dtype=torch.long
        )
        data.append((src_tensor, tgt_tensor))
        
    return data

def convert_text_to_indexes(text, vocab, tokenizer, mode="src"):
    if mode=="src":
        sit = text[0]
        segments = [vocab['<sep>']]
        for s in sit:
            segments += [vocab[token] for token in tokenizer(s.strip("\n"))] + [vocab['<sep>']]
        # 最後消す
        segments[-1] = vocab['<cxt>']
        utt = text[1]
        for u in utt:
            segments += [vocab[token] for token in tokenizer(u.strip("\n"))] + [vocab['<cxt>']]
        return segments
    # 
    elif mode=="tgt":
        return [vocab['<fos>']] + [
            vocab[token] for token in tokenizer(text.strip("\n"))
        ] + [vocab['<eos>']]
    else:
        return []

def convert_text_to_indexes_ntt(text, vocab, tokenizer, mode="src"):
    if mode=="src":
        segments = [vocab['<sep>']]
        for u in text:
            segments += [vocab[token] for token in tokenizer(u.strip("\n"))] + [vocab['<sep>']]
        segments[-1] = vocab['<cxt>']
        return segments
    # 
    elif mode=="tgt":
        return [vocab['<fos>']] + [
            vocab[token] for token in tokenizer(text.strip("\n"))
        ] + [vocab['<eos>']]
    else:
        return []
        

In [130]:
src = src_train_str[0]

In [131]:
train_data = data_process(
    texts_src=src_train_str_, texts_tgt=tgt_train_str,
    vocab_src=vocab_src, vocab_tgt=vocab_tgt,
    tokenizer_src=tokenizer_src, tokenizer_tgt=tokenizer_tgt
)
valid_data = data_process(
    texts_src=src_val_str_, texts_tgt=tgt_val_str_,
    vocab_src=vocab_src, vocab_tgt=vocab_tgt,
    tokenizer_src=tokenizer_src, tokenizer_tgt=tokenizer_tgt
)

In [132]:
print('インデックス化された文章')
print('Input: {}\nOutput: {}'.format(train_data[0][0], train_data[0][1]))
print('')

print('インデックス化された文章をもとに戻す')
print('Input: {}\nOutput: {}'.format(
    ' '.join([vocab_src.itos[x] for x in train_data[0][0]]),
    ' '.join([vocab_tgt.itos[x] for x in train_data[0][1]])
))

インデックス化された文章
Input: tensor([   4,  452,   39, 3440,   22,   19,   16,    7,   33,   12,  232,   22,
          36,    8,  281,   29, 1855,   20,   15,   17,   22,    7,    4,   32,
          52,   15,   17,   11,   26,    7,   33, 1080,  397,   23,  145,  156,
          17,   11,   36,  141, 1103,   13, 2717,   11,   26,   21,    5])
Output: tensor([   2, 4903,   27,   10,    7,  846,    8, 2838,   27,    0,   15,    8,
           0,   13, 3966,   57,   27,  139,   16,    0,   17,    7,    3])

インデックス化された文章をもとに戻す
Input: <sep> 歯医者 って 恐怖 だ よ ね 。 私 の 話 だ けど 、 夫 と 死別 し た ん だ 。 <sep> そう だっ た ん です か 。 私 医療 関係 で 働い でる ん です けど ご 病気 が 原因 です か ? <cxt>
Output: <fos> 難病 で ね 。 ま 、 バスケットボール部 で <unk> た 、 <unk> の 明る さ で 前 に <unk> よ 。 <eos>


In [133]:
batch_size = 128
# batch_size = 48
PAD_IDX = vocab_src['<pad>']
START_IDX = vocab_src['<fos>']
END_IDX = vocab_src['<eos>']

In [134]:
def generate_batch(data_batch):
    
    batch_src, batch_tgt = [], []
    for src, tgt in data_batch:
        batch_src.append(src)
        batch_tgt.append(tgt)
        
    # batch_src = pad_sequence(batch_src, padding_value=PAD_IDX, batch_first=True)
    # batch_tgt = pad_sequence(batch_tgt, padding_value=PAD_IDX, batch_first=True)
    batch_src = pad_sequence(batch_src, padding_value=PAD_IDX, batch_first=False)
    batch_tgt = pad_sequence(batch_tgt, padding_value=PAD_IDX, batch_first=False)
    
    return batch_src, batch_tgt

In [135]:
tgt_val_str_

['そうなんですね。',
 'うーん。',
 'そうなんだ。',
 '夏じゃなく、春の気候のいい頃に帰ってみては?',
 'その人にとっては、嬉しい物だったんだよ。',
 'そうなんですよ、情勢が不安定だったりしますからね。',
 'ごもっともですね。',
 '子沢山のお家だったんですか。',
 'そうですか、私は翻訳家をしていますが、あなたはどんな仕事をしていますか?',
 '銀行員は柵が多そうですね?',
 'お互い大変だよね。',
 'そっか、じゃあ何か楽しい話をしようよ。',
 'わたしはまだ大学院生で、バイトで家庭教師はしています。',
 '商品を返品した方がいいよ。',
 'ラグビーです。',
 '納豆嫌いなんだね。',
 'わたしは絵を描くのが好きなんです。',
 '我慢強いから、もう少し頑張ってみるよ。',
 'すばらしいですね。',
 'そんな深刻にならないで。',
 'わたしは要領が悪いだけで時間がないけどうまくやればできると思うよ。',
 '辰年です。',
 'はい、効果があるという口コミが多いので期待しています。',
 '高級チョコレートだったんだよ!',
 'でしょー!',
 'もちろん、わざとではないですが、娘が大事にしていた玩具なので、うしろめたさを感じます。',
 'いい夫婦の日なんですね。',
 '確かに急なリストラは、日本の会社ではあまりないよね。',
 '学生さんかぁ。',
 '折れにくい上に、書きやすいし、消しやすいし、お値段もそんなに高くないから御勧めだよ!',
 '幼稚園の先生してるよ。',
 '20人くらいかな。',
 'もちろんです!頑張ってください!',
 'わたしは専業主婦なので、悠々自適の生活なんですよ。',
 'もうそんな大きくなっちゃったの?',
 'うん。',
 '誕生日が近かったので、わたし宛てのプレゼントかなって期待してしまいました。',
 'そんなに混んでるんだね。',
 '僧侶さんでしたか!',
 'いいなぁ。',
 'まずは記帳してみましょう',
 'まあまあ、汚れに気づいた時に掃除すれば大丈夫、大丈夫。',
 'そうでしたか、それはご愁傷様です。',
 'マナーは守ってほしいですね。',
 'そうなんですか、外国の方だと気付かなかったんです。',
 'そうですよねー!',
 'うん!',
 '私

In [136]:
train_iter = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(valid_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)

In [137]:
def train(model, data, optimizer, criterion, PAD_IDX):
    
    model.train()
    losses = 0
    for src, tgt in tqdm(data):
        
        src = src.to(device)
        tgt = tgt.to(device)

        input_tgt = tgt[:-1, :]

        mask_src, mask_tgt, padding_mask_src, padding_mask_tgt = create_mask(src, input_tgt, PAD_IDX)

        # print(src.shape, tgt.shape)

        logits = model(
            src=src, tgt=input_tgt,
            mask_src=mask_src, mask_tgt=mask_tgt,
            padding_mask_src=padding_mask_src, padding_mask_tgt=padding_mask_tgt,
            memory_key_padding_mask=padding_mask_src
        )

        optimizer.zero_grad()

        output_tgt = tgt[1:, :]
        loss = criterion(logits.reshape(-1, logits.shape[-1]), output_tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

        del logits
        del loss
        
    return losses / len(data)


def evaluate(model, data, criterion, PAD_IDX):
    
    model.eval()
    losses = 0
    for src, tgt in data:
        
        src = src.to(device)
        tgt = tgt.to(device)

        input_tgt = tgt[:-1, :]

        mask_src, mask_tgt, padding_mask_src, padding_mask_tgt = create_mask(src, input_tgt, PAD_IDX)

        logits = model(
            src=src, tgt=input_tgt,
            mask_src=mask_src, mask_tgt=mask_tgt,
            padding_mask_src=padding_mask_src, padding_mask_tgt=padding_mask_tgt,
            memory_key_padding_mask=padding_mask_src
        )
        
        output_tgt = tgt[1:, :]
        loss = criterion(logits.reshape(-1, logits.shape[-1]), output_tgt.reshape(-1))
        losses += loss.item()

        del logits
        del loss
        
    return losses / len(data)

In [138]:
vocab_size_src = len(vocab_src)
vocab_size_tgt = len(vocab_tgt)
embedding_size = 320
nhead = 8
dim_feedforward = 100
num_encoder_layers = 4
num_decoder_layers = 4
dropout = 0.1

model = Seq2SeqTransformer(
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    embedding_size=embedding_size,
    vocab_size_src=vocab_size_src, vocab_size_tgt=vocab_size_tgt,
    dim_feedforward=dim_feedforward,
    dropout=dropout, nhead=nhead,
    PAD_IDX=PAD_IDX
)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# lr = 0.05
optimizer = torch.optim.Adam(model.parameters())
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [139]:

import time

epoch = 30
best_loss = float('Inf')
best_model = None
patience = 10
counter = 0

for loop in range(1, epoch + 1):
    
    start_time = time.time()
    
    loss_train = train(
        model=model, data=train_iter, optimizer=optimizer,
        criterion=criterion, PAD_IDX=PAD_IDX
    )
    
    elapsed_time = time.time() - start_time
    
    loss_valid = evaluate(
        model=model, data=valid_iter, criterion=criterion, PAD_IDX=PAD_IDX
    )
    
    print('[{}/{}] train loss: {:.2f}, valid loss: {:.2f}  [{}{:.0f}s] count: {}, {}'.format(
        loop, epoch,
        loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        counter,
        '**' if best_loss > loss_valid else ''
    ))
    
    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = model
        counter = 0
        
    if counter > patience:
        break
    
    counter += 1

100%|██████████| 40/40 [00:08<00:00,  4.84it/s]


[1/30] train loss: 5.95, valid loss: 5.13  [8s] count: 0, **


100%|██████████| 40/40 [00:07<00:00,  5.06it/s]


[2/30] train loss: 5.53, valid loss: 5.11  [8s] count: 1, **


100%|██████████| 40/40 [00:07<00:00,  5.10it/s]


...

[11/30] train loss: 5.41, valid loss: 5.94  [8s] count: 9, 


100%|██████████| 40/40 [00:08<00:00,  4.81it/s]


[12/30] train loss: 5.41, valid loss: 5.86  [8s] count: 10, 


100%|██████████| 40/40 [00:08<00:00,  4.89it/s]


[13/30] train loss: 5.41, valid loss: 6.46  [8s] count: 11, 


In [140]:
model_path = "../models/transformer/"
model_name = "CModel_lim={0}_best.pickle".format(lim)
modelM = DataManager(model_path)
modelM.save_data(model_name, best_model)
model_name = "CModel_lim={0}.pickle".format(lim)
modelM.save_data(model_name, model)

success save : ../models/transformer/CModel_lim=5000_best.pickle
success save : ../models/transformer/CModel_lim=5000_best.pickle
success save : ../models/transformer/CModel_lim=5000.pickle
success save : ../models/transformer/CModel_lim=5000.pickle
